In [68]:
import numpy as np
import json
import glob
import pandas as pd
import csv

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy
import spacy
from nltk.corpus import stopwords

# Vis
import pyLDAvis
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [69]:
all_files = ["coinbase.csv", "binance.csv", "ftx.csv", "kraken.csv"]

In [70]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = pd.read_csv(file)
        # print(data)
        # data = json.load(f)
    return (data);

def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4);

# Combine all the csv files into one

li = []
for filename in all_files:
    df = load_data(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
data = frame.loc[:, ["content", "score"]]
data = data.loc[(data.score <= 2)] # Grab only poor reviews
content = json.loads(data.loc[:, "content"].to_json())

# Prefilter step

# Remove reviews with <= 50 chars
data = [content[key] for key in content if len(content[key]) > 50];
len(data)

## Break each sentence into its own document

#

5543

In [71]:
# Stop words are basically words that are not really useful in the topic model
# i.e. words that you want to eliminate from consideration
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [72]:
# data = load_data("ushmm_dn.json")["texts"]
print(data[0][0:90]) # Prints out first text, char 0-89

DO NOT RECOMMEND - They have no problem taking your money to buy crypto, but then mistaken


In [73]:
# Reducing words to their root form (lemma) using spacy or nltk to grab roots
# Specify allowed_postags for spacy

# Takes a list of texts i.e. [text1, text2...] and returns lemmatised [text1..]
# Allowed_post_tages -> Basically default arguments in python 
# This is a list initalised that is used to filter tokens 
# pos_ tag marks category of word token belongs to i.e. only append nouns,adj,verbs,and adverbs 
# e.g. conjunctions (but, and, because), and pronouns (he/she) is ignored
def lemmatization(texts, allowed_postags=["NOUN", "ADJ"]):

    # Spacy load creates an an nlp parser i.e. function pointer 
    spacy.load
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []

    for text in texts:
        doc = nlp(text) # Create document from the raw text/string
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags and token.pos_ not in stopwords:
               new_text.append(token.lemma_) # Appends only lemma to new_text
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out);

lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:90])

problem money crypto suspicious customer service flat problem problem money access smart g


In [74]:
# Simple preprocess converts documents to list of lower case tokens ignoring ones
# which are too short or too long.
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True);
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])


['problem', 'money', 'crypto', 'suspicious', 'customer', 'service', 'flat', 'problem', 'problem', 'money', 'access', 'smart', 'guy', 'form', 'theft', 'good', 'luck', 'clown']


In [75]:
# BIGRAMS AND TRIGRAMS

# min_count - ignore all words/bigrams with total collected count lower than this value.
# threshold - score for forming phrases play around with this -> related to scoring function
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=1) 

# First arg checks ensures no overlap between bigraph phrases and trigram phrases.
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=1)

# Create function pointer i.e. phraser that converts document to list of bigram
bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# print(data_bigrams_trigrams[0])

# new_words = []
# for word in data_bigrams_trigrams[0]:
#     if '_' in word:
#         new_words.append(word)
# print(data_bigrams)


In [76]:
# TF-IDF Removal - to remove words that are not important to the topic
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

# Goes through words and looks at words that occur too frequentely and removes them
# Corpus becomes new corpus where those irrelevant words are removed
# low_value = 0.03
low_value = 0.2 # TODO
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow




[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 3), (10, 1), (11, 1), (12, 1)]


In [77]:
# id2word = corpora.Dictionary(data_words)
# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text) # Create a bag of words all (word: f pairs) for 1 doc
#     corpus.append(new)

# print(corpus[0][0:20])
# #(0,2) - 0 = index of word in the dictionary, 2 = frequency of word in that text

# word = id2word[[0][:1][0]] # i.e. text 1 -> first tuple -> first value in tuple
# print(word) # i.e. able is the first element in teh dictionary 

In [78]:
lda_model = gensim.models.LdaModel(corpus=corpus
                                    ,id2word=id2word
                                    ,num_topics=6
                                    ,random_state=100
                                    ,update_every=1
                                    ,chunksize=100
                                    ,passes=10
                                    ,alpha="auto")

### Idea
I think I can separate the review data set into one star vs 5 star reviews and look at if there are any differences in the topics that are mentioned.

In [79]:
# Visualising the data
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/root/miniconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: Deprecatio

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.115467  0.307172       1        1  20.431265
5     -0.110827 -0.319293       2        1  17.650385
0     -0.326697 -0.060285       3        1  16.270991
3     -0.215511  0.264198       4        1  15.638503
1      0.222138 -0.250017       5        1  15.390729
2      0.315429  0.058225       6        1  14.618128, topic_info=              Term        Freq       Total Category  logprob  loglift
74         account  374.000000  374.000000  Default  30.0000  30.0000
23             app  327.000000  327.000000  Default  29.0000  29.0000
759           shib  265.000000  265.000000  Default  28.0000  28.0000
97             fee  238.000000  238.000000  Default  27.0000  27.0000
2           crypto  246.000000  246.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
239       annoying   39.553041   40.339257   Topic6  -4.8383   1.9032
676      different   38.160174   38.948577   Topic6  -4.8741   1.9025
51       developer   36.312934   37.100081   Topic6  -4.9237   1.9015
3963       listing   50.253276   51.343564   Topic6  -4.5988   1.9014
1153  app_unusable   34.412628   35.208484   Topic6  -4.9775   1.9000

[214 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
447       3  0.989335        able
74        5  0.998396     account
284       5  0.991253     address
62        4  0.974714      amount
239       6  0.991590    annoying
...     ...       ...         ...
436       3  0.983613        well
1410      6  0.990221       whole
667       1  0.983937  withdrawal
1291      4  0.993570        word
92        1  0.989154       wrong

[189 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 1, 4, 2, 3])

In [80]:
# Further preprocessing


def pre_process_unseen(unseen_document, useless_words=["app", "shib", "day", "time", "kraken", "hour", "money"]):
    words = " ".join(lemmatization([unseen_document]))
    cleaned_words = [" ".join(gen_words([w])[0])]
    cleaned_words = [word for word in cleaned_words if word not in useless_words]
    return cleaned_words


In [83]:
labels = {1: "transaction", 2: "trust", 3: "fee", 4: "idk", 5: "idk", 6: "app"}
for i in range(len(lda_model.get_topics())):
    print(lda_model.print_topic(i, 5))

0.029*"bank" + 0.024*"platform" + 0.020*"problem" + 0.020*"code" + 0.020*"deposit"
0.071*"account" + 0.038*"promise" + 0.035*"company" + 0.033*"star" + 0.026*"update"
0.048*"fee" + 0.033*"coin" + 0.030*"transaction" + 0.028*"verification" + 0.023*"shiba"
0.049*"shib" + 0.028*"option" + 0.025*"error" + 0.024*"good" + 0.022*"people"
0.031*"money" + 0.021*"website" + 0.021*"portfolio" + 0.021*"way" + 0.019*"hour"
0.054*"app" + 0.041*"crypto" + 0.035*"kraken" + 0.032*"day" + 0.031*"time"


In [89]:
unseen_document = "The fee is too high"
bow_vector = id2word.doc2bow(pre_process_unseen(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    # lda_mode[bow_vector] % gets the topic probability for a document
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
# TODO: Think about how to classify using 

Score: 0.22350287437438965	 Topic: 0.029*"bank" + 0.024*"platform" + 0.020*"problem" + 0.020*"code" + 0.020*"deposit"
Score: 0.18693907558918	 Topic: 0.031*"money" + 0.021*"website" + 0.021*"portfolio" + 0.021*"way" + 0.019*"hour"
Score: 0.16559089720249176	 Topic: 0.054*"app" + 0.041*"crypto" + 0.035*"kraken" + 0.032*"day" + 0.031*"time"
Score: 0.14564518630504608	 Topic: 0.071*"account" + 0.038*"promise" + 0.035*"company" + 0.033*"star" + 0.026*"update"
Score: 0.14540106058120728	 Topic: 0.049*"shib" + 0.028*"option" + 0.025*"error" + 0.024*"good" + 0.022*"people"
Score: 0.13292092084884644	 Topic: 0.048*"fee" + 0.033*"coin" + 0.030*"transaction" + 0.028*"verification" + 0.023*"shiba"


In [ ]:
# Save the model into memory
lda_model.save(f"models/{csv_file[:-4]}.model");

NameError: name 'csv_file' is not defined